## With Quantum Circuits

In [1]:
import pyqcs
from pyqcs import R, X, H

def expIphiZ(act, phi):
    gates = pyqcs.R(act, -phi)|pyqcs.X(act)|pyqcs.R(act, phi)|pyqcs.X(act)
    return gates
    

def su2_to_circuit(act,param):
    phi0, phi1, phi2 = param
    th0 = phi0
    th1 = 1/2*(phi1 + phi2)
    th2 = 1/2*(phi1 - phi2)
    S = pyqcs.R(act, np.pi/2)
    Sd = pyqcs.R(act, -np.pi/2)
    gates = []
    gates.append(expIphiZ(act,th2))
    gates.append(Sd|pyqcs.H(act)|expIphiZ(act,th0)|pyqcs.H(act)|S)
    gates.append(expIphiZ(act,th1))
    
    return pyqcs.list_to_circuit(gates)



In [2]:
#testing 
def test_expIaZ():
    print("testing ExpIaZ:")
    for i in range(10000):
        phi = np.random.rand()*2*np.pi
        nbits = 1
        psi0 = pyqcs.State.new_zero_state(nbits)
        psi1 = X(0)*psi0
        expZ = expIphiZ(0, phi)
        psi0out = expZ*psi0
        psi1out = expZ*psi1
        val0 = psi0out._qm_state[0]
        val1 = psi1out._qm_state[1]

        val0r = np.exp(1j*phi)
        val1r = np.exp(-1j*phi)
        if not np.isclose(val0r, val0):
            raise ValueError("wrong for phi={}".format(phi))

        if not np.isclose(val1r, val1):
            raise ValueError("wrong for phi={}".format(phi))
    print("everything went fine!")
    
def test_su2():
    print("testing su2")
    for i in range(10000):
        param = np.random.rand(3)*np.pi*2
        U_circ = su2_to_circuit(0, param)
        U = su2(param)

        psi0 = pyqcs.State.new_zero_state(nbits)
        psi1 = X(0)*psi0

        Psi0 = np.matrix([1,0]).T
        Psi1 = np.matrix([0,1]).T

        psi0out = U_circ*psi0
        Psi0out = U@Psi0
        psi1out = U_circ*psi1
        Psi1out = U@Psi1

        for i in range(2):
            val0 = psi0out._qm_state[i]
            Val0r = np.complex(Psi0out[i])
            if not np.isclose(val0, Val0r):
                raise ValueError("error: {}".format(val0-Val0r))

            val1 = psi1out._qm_state[i]
            Val1r = np.complex(Psi1out[i])
            if not np.isclose(val0, Val0r):
                raise ValueError("error: {}".format(val1-Val1r))

    print("everything went fine!")

In [5]:
def bootstrap(data, n_boot=5):
    means = np.zeros(n_boot, dtype=np.double)
    for i in range(n_boot):
        tmp = np.random.choice(data, size=len(data), replace=True)
        means[i] = np.mean(tmp)
        
    return np.std(means, ddof=1)



class PathTree:
    def __init__(self, prob, circs, initial_st):
        self.size = len(prob)
        self.prob = prob
        self.circs = circs
        self.initial_st = initial_st
        
    def walk_a_path(self, n_measure=1000):
        drawn_circs = np.zeros(self.size, dtype=int)
        for i in range(self.size):
            q = np.random.rand()

            if q < self.prob[i]:
                drawn_circs[i]=0
            else:
                drawn_circs[i]=1

        psi = self.initial_st
        for i,d in enumerate(drawn_circs):
            psi = self.circs[i,d]*psi
            
        dict_res = pyqcs.sample(psi, 1, n_measure)
        res = np.zeros(2, dtype=np.double)
        for key, value in dict_res.items():
            res[key] = value/n_measure
        return drawn_circs, res
    
    def sample_walks(self, n_samples=100, n_measure=1000, logging=False):
        result = np.zeros((n_samples, 2), dtype=np.double)
        
        if logging:
            file = open("log.txt", "w")
        
        for i in range(n_samples):
            drawn_circs, res = self.walk_a_path(n_measure=n_measure)
            if logging:
                for d in drawn_circs:
                    file.write("%d"%d)
                file.write("\n")
            result[i,:] = res
        
        prob_0 = [np.mean(result[:,0]), bootstrap(result[:,0])]
        prob_1 = [np.mean(result[:,1]), bootstrap(result[:,1])]
            
        return prob_0, prob_1

In [6]:
def evolution_quantum_circuit(data):
    n_t = data.shape[0]
    probs = data[:,0]
    circs = np.ndarray((n_t,2), dtype=pyqcs.gates.circuits.AnonymousCompoundGateCircuit)
    
    for i, d in enumerate(data):
        param0 = d[1:4]
        param1 = d[4:]
        
        U0circ = su2_to_circuit(0, param0)
        U1circ = su2_to_circuit(0, param1)
        
        circs[i,:]=[U0circ, U1circ]
    
    
    QC_probs = np.zeros((n_t, 2, 2), dtype=np.double)
    
    init_st = pyqcs.State.new_zero_state(1)
    print("init_state: ", init_st)
    for i in range(n_t):
        print("#", end="", flush=True)
        tree = PathTree(probs[:i+1], circs[:i+1], init_st)
        p0, p1 = tree.sample_walks(logging=True)
        QC_probs[i,0,:] = p0
        QC_probs[i,1,:] = p1
        
    return QC_probs

In [7]:
def evolution_density_matrix(n_t, dt):
    rho_0 = np.matrix([[1,0],[0,0]], dtype=np.complex)
    DM_probs = np.zeros((n_t,2), dtype=np.double)
    for i in range(n_t):
        Ks = op.get_kraus_(T**(i+1))
        rho_out = op.channel_(rho_0, Ks)
        DM_probs[i,:] = [rho_out[0,0].real, rho_out[1,1].real]
        
    return DM_probs

DM_probs = evolution_density_matrix(data.shape[0], dt)

NameError: name 'data' is not defined

In [8]:
times = np.arange(QC_probs.shape[0])*dt

colors = ["tab:olive", "tab:blue"]
labels = ["|0>", "|1>"]

plt.figure(figsize=(20,10))

for i in range(2):
    plt.plot(times, DM_probs[:,i], color=colors[i], label=labels[i])
    plt.errorbar(times, QC_probs[:,i,0], yerr=QC_probs[:,i,1], fmt="o", color=colors[i])
plt.xlabel("$t$")
plt.ylabel("$p$")
plt.legend()
plt.show()

NameError: name 'np' is not defined